<img src="./img/ov_logo.png" style="width: 30%" />
<img src="./img/anvil_logo.png" style="width: 80%" />

### Build a trusted machine-to-machine negotiation app in 5 minutes

ANVIL bridges Fetch.AI and Sovrin, bringing trusted agents to the Open Economic Framework.  View and edit code built with the ANVIL API live: learn how to build trusted, automated settlement between bots from scratch in just 5 minutes.

### Step 1: import the ANVIL API

In [ ]:
from fetch.agents import search, offer_service, purchase_service
from sovrin.utilities import generate_base58
from sovrin.setup import setup_pool, set_self_up, teardown
from sovrin.onboarding import demo_onboard
from sovrin.schema import create_schema, create_credential_definition
from sovrin.credentials import offer_credential, receive_credential_offer, request_credential, create_and_send_credential, store_credential
from sovrin.proofs import request_proof_of_credential, create_proof_of_credential, verify_proof
import json
print('ANVIL API imported')

### Step 2: set up agents

We are going to run through the [certified charging station use case](https://outlierventures.io/research/anvil-a-fetch-ai-and-sovrin-bridge/).

This involves three agents: a manufacturer (issuer), charging station (prover) and electric car (verifier).


<img src="./img/manufacturer_icon.png" style="width: 10%; display: inline-block" />
<div style="width: 10%; height: 1px; display: inline-block"></div>
<img src="./img/charge_icon.png" style="width: 10%; display: inline-block; display: inline-block" />
<div style="width: 10%; height: 1px; display: inline-block"></div>
<img src="./img/car_icon.png" style="width: 10%; display: inline-block" />

We will:
1. Have the manufacturer issue a credential to the charging station
2. Have the charging station offer a charging service on Fetch.AI
3. Use an electric car to verify the charging station is genuine

Let's set up our Sovrin node pool and agents, establishing secure channels between them:

In [ ]:
pool_name, pool_handle = await setup_pool('local')

steward = await set_self_up('steward', generate_base58(64), generate_base58(64), pool_handle, seed = '000000000000000000000000Steward1')
issuer = await set_self_up('issuer', generate_base58(64), generate_base58(64), pool_handle)
prover = await set_self_up('prover', generate_base58(64), generate_base58(64), pool_handle)
verifier = await set_self_up('verifier', generate_base58(64), generate_base58(64), pool_handle)

steward, issuer = await demo_onboard(steward, issuer)
steward, verifier = await demo_onboard(steward, verifier)
issuer, prover = await demo_onboard(issuer, prover)
verifier, prover = await demo_onboard(verifier, prover)

### Step 3: issue a credential

#### Part A

We define a _schema_ (which attributes the credential should have), and offer this to the charging station:

In [ ]:
schema = {
    "name": "Charging Station Certification",
    "version": "1.0",
    "attributes": ["output", "firmware", "grounded", "license", "year", "id"]
}
unique_schema_name, schema_id, issuer = await create_schema(schema, issuer)
issuer = await create_credential_definition(issuer, schema_id, unique_schema_name, revocable = False)
issuer, cred_offer = await offer_credential(issuer, unique_schema_name)

#### Part B

The charging station receives the credential offer, and fills in its values for the credential attributes as a _credential request_:

In [ ]:
prover['authcrypted_cred_offer'] = cred_offer
prover = await receive_credential_offer(prover)
cred_request = json.dumps({
    "output": {"raw": "22kW", "encoded": "22"},
    "grounded": {"raw": "yes", "encoded": "1"},
    "license": {"raw": "Grounded Station", "encoded": "145"},
    "firmware": {"raw": "2.1", "encoded": "45"},
    "year": {"raw": "2019", "encoded": "2019"},
    "id": {"raw": "did:ov:37h3r", "encoded": "3708318"}
})
prover, cred_request = await request_credential(prover, cred_request)

#### Part C

<img src="./img/cred.png" style="width: 20%; display: inline-block;" />

The manufacturer receives the credential request, creates the final credential and sends it back:

In [ ]:
issuer['authcrypted_cred_request'] = cred_request
issuer, cred = await create_and_send_credential(issuer)

#### Part D

The charging station receives the credential and stores it in its wallet:

In [ ]:
prover['authcrypted_cred'] = cred
prover = await store_credential(prover)

### Step 4: offer a charging service

Services are offered using a path to the Fetch.AI data model describing it. This is a set of JSON files, and for a charging service could be as follows.
Data model:
```JSON
{
    "name" : "Car charging",
    "description": "Electric car charging service.",
    "attributes": {
        "license": "Grounded Station",
        "output": "22kW"
    }
}
```
Service description:
```JSON
{
    "license": true,
    "output": true
}
```
Data to send:
```JSON
{
    "output": "22kW",
    "duration": "1hr",
    "start": "1min"
}
```
For the sake of this demo we store these files at `./charging_service`.

We offer charging at a price of 100 Fetch.AI tokens:

In [18]:
offer_service(100, '../../charging_service')

### Step 5: verify the charging station is genuine

#### Part A
The car approaches the charging pole and requests a proof of credential:

In [ ]:
proof_request = json.dumps({
    "nonce": "0123456789012345678901234",
    "name": "Grounded Station proof",
    "version": "0.1",
    "requested_attributes": {
        "attr1_referent": {
            "name": "output"
        },
        "attr2_referent": {
            "name": "grounded"
        },
        "attr3_referent": {
            "name": "license"
        },
        "attr4_referent": {
            "name": "firmware"
        },
        "attr5_referent": {
            "name": "id"
        }
    },
    "requested_predicates": {
        "predicate1_referent": {
            "name": "year",
            "p_type": ">=",
            "p_value": 2019
        }
    }
})
verifier, proof_request = await request_proof_of_credential(verifier, proof_request)

#### Part B

The charging pole receives the proof request and constructs a proof. It specifies what it will attest itself as well as what to get from the Sovrin ledger:

In [21]:
prover['authcrypted_proof_request'] = proof_request
self_attested_attributes = {
    "attr1_referent": "22kW",
    "attr5_referent": "did:ov:37h3r"
}
prover, proof = await create_proof_of_credential(prover, self_attested_attributes, requested_attrs = [2,3,4], requested_preds = [1], non_issuer_attributes = [])

NameError: name 'proof_request' is not defined

#### Part C
The car receives the proof and verifies it using the Sovrin ledger, specifying any desired assertions to make on the attributes:

In [ ]:
verifier['authcrypted_proof'] = proof
assertions_to_make = {
    "revealed": {
        "attr2_referent": "yes",
        "attr3_referent": "Grounded Station",
        "attr4_referent": "2.1"
    },
    "self_attested": {
        "attr1_referent": "22kW",
        "attr5_referent": "did:ov:37h3r"
    }
}
verifier = await verify_proof(verifier, assertions_to_make)

### Step 6: Purchase the service
Having verified that the charging station is genuine, the car purchases the charging service, specifying the maximum price it is willing to pay and the properties it desires (`license` and `output`):

In [ ]:
purchase_service(101, 'license_output')

Settlement is automatic, and the charging starts. The transactions is automatically logged to the Fetch.AI _smart ledger_.

Finally, tear down connections:

In [ ]:
await teardown(pool_name, pool_handle, [steward, issuer, prover, verifier])

Now you know how to build a trusted machine-to-machine negotiation app, so [get building!](https://www.github.com/OutlierVentures/ANVIL)